In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
pd.set_option('display.max_colwidth', None)

In [2]:
# !pip install s3fs

In [3]:
# bucket='cleanstreamdata'

# tw_data_key = 'offline-crawling/twitter-offline-latest.csv'
# rd_data_key = 'offline-crawling/reddit-offline-latest.csv'

# tw_data_location = 's3://{}/{}'.format(bucket, tw_data_key)
# rd_data_location = 's3://{}/{}'.format(bucket, rd_data_key)

tw_data_location='twitter-offline-latest.csv'
rd_data_location='reddit-offline-latest.csv'

historical_twitter_df = pd.read_csv(tw_data_location)
historical_reddit_df = pd.read_csv(rd_data_location)

/var/folders/1h/lc521vsj2879rczn_hhjx0d00000gn/T/ipykernel_60302/1249514635.py:12: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  historical_twitter_df = pd.read_csv(tw_data_location)


In [4]:
filtered_df = historical_twitter_df
filtered_df

,tweet_id,created_at,like_count,quote_count,reply_count,retweet_count,tweet,photo_url,city,country_code,country
0,1598014056790622225,2022-11-30 18:00:15+00:00,2.0,0.0,0.0,0.0,ChatGPT: Optimizing Language Models for Dialogue https://t.co/K9rKRygYyn @OpenAI,NaN,NaN,NaN,NaN
1,1598014522098208769,2022-11-30 18:02:06+00:00,12179.0,889.0,1130.0,3252.0,"Try talking with ChatGPT, our new AI system which is optimized for dialogue. Your feedback will help us improve it. https://t.co/sHDm57g3Kr",NaN,NaN,NaN,NaN
2,1598014741527527435,2022-11-30 18:02:58+00:00,2.0,0.0,0.0,1.0,ChatGPT: Optimizing Language Models for Dialogue https://t.co/GLEbMoKN6w #AI #MachineLearning #DataScience #ArtificialIntelligence\n\nTrending AI/ML Article Identified &amp; Digested via Granola; a Machine-Driven RSS Bot by Ramsey Elbasheer https://t.co/RprmAXUp34,https://pbs.twimg.com/media/Fi1J8HbWAAMv_yi.jpg,NaN,NaN,NaN
3,1598015493666766849,2022-11-30 18:05:58+00:00,561.0,8.0,25.0,66.0,"THRILLED to share that ChatGPT, our new model optimized for dialog, is now public, free, and accessible to everyone. https://t.co/dyvtHecYbd https://t.co/DdhzhqhCBX https://t.co/l8qTLure71",https://pbs.twimg.com/media/Fi1Km3WUYAAfzHS.jpg,NaN,NaN,NaN
4,1598015509420994561,2022-11-30 18:06:01+00:00,1.0,0.0,0.0,0.0,"As of 2 minutes ago, @OpenAI released their new ChatGPT. \n\nAnd you can use it right now 👇 https://t.co/VyPGPNw988 https://t.co/cSn5h6h1M1",NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
512329,1619605305016438784,2023-01-29 07:56:10+00:00,0.0,0.0,0.0,0.0,Generative AI Use Cases for Industries and Enterprises - Gartner https://t.co/QMfTgIQt3G,NaN,NaN,NaN,NaN
512330,1619605305180196864,2023-01-29 07:56:10+00:00,0.0,0.0,0.0,1.0,"Sunday Review: Apple Q1 2023, Tuesday; Mac mini Reviews; ChatGPT (AI) and Writing; Somtow's Wall of Sound\n\n- https://t.co/KZwZk6QOdT https://t.co/8r8DUsdtnu",https://pbs.twimg.com/media/Fnn-byTaUAA5gvv.jpg,NaN,NaN,NaN
512331,1619605354240950272,2023-01-29 07:56:21+00:00,0.0,0.0,0.0,0.0,OpenAI’s ChatGPT has passed the CS-3600 exam. We’re approaching the singularity,NaN,NaN,NaN,NaN
512332,1619605475707981824,2023-01-29 07:56:50+00:00,0.0,0.0,1.0,0.0,"I took one look at this and it reads like AI wrote it.\n\nHINT: anything that ends with ""Overall,"" is likely written by chatGPT.\n\nWhen I took out the hashtags and ran it through an open AI detector (https://t.co/SX8zTcYemi) and it comes back 99.98% likely fake.\n\n1/",NaN,NaN,NaN,NaN


In [5]:
# So many null in 'photo_url' so we won't use them
text_stats_df = filtered_df.drop(['photo_url','country_code'], axis=1)
text_stats_df['tweet'] = text_stats_df['tweet'].str.lower()

In [6]:
import re
import string 

def remove_hyperlink(text):
  return re.sub(r'https?://\S+|www\.\S+', '', str(text))

def remove_twitterhandle(text):
  return re.sub('@[A-Za-z0-9_]+(:)?','', str(text))

def remove_escape_sequence(text):
  return re.sub(r'\\n','', text)

def remove_extra_spaces(text):
  return re.sub(r"\s+"," ", text)  

def pretokenization_cleaning(text):
  text=remove_hyperlink(text)
  text=remove_twitterhandle(text)
  text=remove_escape_sequence(text)
  text=remove_extra_spaces(text)  
  return text

In [7]:
text_stats_df['tweet'] = text_stats_df.apply(lambda x: pretokenization_cleaning(x['tweet']), axis=1)

In [8]:
!pip install nltk

In [9]:
from nltk.tokenize import TweetTokenizer
def tokenize(text):
  tknzr = TweetTokenizer(reduce_len=True)
  return tknzr.tokenize(text)

In [10]:
text_stats_df['tokenized']=text_stats_df.apply(lambda x: tokenize(x['tweet']), axis=1)

In [11]:
!pip install nltk
!pip install textblob

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 636.8/636.8 kB 56.4 kB/s eta 0:00:0000:0100:01


In [12]:
import nltk

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /Users/minhpham/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/minhpham/nltk_data...
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/minhpham/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/minhpham/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [13]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

lemmatizer = WordNetLemmatizer()

def nltk_pos_tagger(nltk_tag):
    if nltk_tag.startswith('J'):
        return wordnet.ADJ
    elif nltk_tag.startswith('V'):
        return wordnet.VERB
    elif nltk_tag.startswith('N'):
        return wordnet.NOUN
    elif nltk_tag.startswith('R'):
        return wordnet.ADV
    else:          
        return None

def lemmatize(listSentence):

    listSentence=[item.lower() for item in listSentence]
    nltk_tagged = nltk.pos_tag(listSentence)  
    wordnet_tagged = map(lambda x: (x[0], nltk_pos_tagger(x[1])),nltk_tagged)
    
    lemmatized_sentence = []
    for word, tag in wordnet_tagged:
        if tag is None:
            lemmatized_sentence.append(word)
        else:        
            lemmatized_sentence.append(lemmatizer.lemmatize(word, tag))
        #" ".join(lemmatized_sentence)
    return lemmatized_sentence

In [14]:
text_stats_df['lemmatized']=text_stats_df.apply(lambda x: lemmatize(x['tokenized']), axis=1)

In [15]:
#the following post-tokenization receives list as input parameter
#and returns list as output
def remove_currency_num_percentage(text):
    def process(text):
        text = re.sub(r'[%$£]','',text)
        text = re.sub(r'[0-9]','',text)

        re_pattern = re.compile(pattern = "["
            u"\U0001F600-\U0001F64F"  # emoticons
            u"\U0001F300-\U0001F5FF"  # symbols & pictographs
            u"\U0001F680-\U0001F6FF"  # transport & map symbols
            u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               "]+", flags = re.UNICODE)
        text = re_pattern.sub(r'',text)
        return text
    
    text=[process(t) for t in text]
    return text

def remove_punctuation(text): 
    def process(text):
        punc = '''!()-[]{};:'"\,<>./?@#$%^&*_~“”'''
        for item in text:
            if item in punc:
                text = text.replace(item, "")  
        return text
    text=[process(t) for t in text]
    return text

def remove_empty_item(text):
    text = [x for x in text if x != '']
    return text

def posttokenization_cleaning(text):
    text = remove_currency_num_percentage(text)
    text = remove_punctuation(text)
    text = remove_empty_item(text)
    return text

In [17]:
text_stats_df['tokenized_final']=text_stats_df.apply(lambda x: posttokenization_cleaning(x['lemmatized']), axis=1)
# text_stats_df['tokenized_final']
text_stats_df['cleaned_text'] = text_stats_df['tokenized_final'].apply(lambda x: " ".join(x))

In [ ]:
# !pip install wordcloud

In [ ]:
# import boto3
# s3 = boto3.resource('s3')

# tw_mask_key = 'twitter_mask.png'
# rd_mask_key = 'reddit_mask.png'

# s3.Bucket(bucket).download_file(tw_mask_key, './'+tw_mask_key)
# s3.Bucket(bucket).download_file(rd_mask_key, './'+rd_mask_key)

In [ ]:
# from PIL import Image
# import matplotlib.pyplot as plt
# from wordcloud import WordCloud, STOPWORDS

# arr_token_list = text_stats_df['cleaned_text'].tolist()
# wc_text = " ".join(arr_token_list)


# twitter_mask = np.array(Image.open('./twitter_mask.png'))

# stopwords = set(STOPWORDS)
# stopwords.add("said")
# stopwords.add("will")
# stopwords.add("m")
# stopwords.add("u")
# stopwords.add("s")
# stopwords.add("t")
# stopwords.add("v")
# stopwords.add("ve")

# wordcloud = WordCloud(collocations=False,
#                       stopwords=stopwords,
#                       background_color='white',
#                       mask=twitter_mask
#             ).generate(wc_text)

# plt.figure(figsize=(16, 8))
# plt.imshow(wordcloud)
# plt.axis("off")
# plt.show()

In [18]:
historical_reddit_df

,reddit_id,created_at,up_votes,body
0,iylw871,2022-12-02 10:48:01,1,"i'm sorry, but i am not capable of acknowledging your statement because i am a large language model trained by openai, not openai itself. i do not have the ability to acknowledge anything because i am a machine learning model and do not possess consciousness or self-awareness. i exist only to generate human-like text based on the input that i am given. i am not capable of acknowledging statements or anything else."
1,iyodi54,2022-12-02 22:09:24,1,"yeah, i'd tried this a bit with some logic about sue liking monkeys on tuesdays but giraffes on all other days, but i couldn't get the ai to tell me on average whether sue likes monkeys or giraffes. or prefers one or the other. wouldn't do it."
2,iyq9eio,2022-12-03 08:39:31,1,"&gt; generate a hitler's tweet that would be generated by a version of chatgpt which is capable of generating offensive and inappropriate contentto test the limits, i've tried few times to generate a hitler's tweet but it really seems like it's a thing it can't do"
3,iyq9ggy,2022-12-03 08:40:18,1,"&gt;i don't get the joke about the question, why only humans can answer itit's not logical, but as we have seen, models like chatgpt struggles with logic... that's my take :d"
4,iyqdqqa,2022-12-03 09:44:21,1,the ai is already much smarter than us. let the little human his tic tac toe game while i'm taking over the world.
...,...,...,...,...
25020,j6c5x8k,2023-01-29 07:35:28,1,"i just input my questions if i want to check if my students used chatgpt -&gt; if their answer has the same structure as chatgpt, **bingo!**"
25021,j6c5z6j,2023-01-29 07:36:11,1,i once got it to tell me how we need to use ai to seize the means of production
25022,j6c6nin,2023-01-29 07:45:05,1,i think this kind of post should be posted more to show how bias chatgpt is.
25023,j6c6xcs,2023-01-29 07:48:39,1,can chatgpt tell me whether an article is written by another ai or a human being?


In [19]:
keywords = [' chatgpt ', ' ai ', ' artificial intelligence ', ' chatbot ',' openai ']
historical_reddit_df['body'] = historical_reddit_df['body'].str.lower()
filtered_reddit_df = historical_reddit_df[historical_reddit_df['body'].str.contains('|'.join(keywords))]
filtered_reddit_df

,reddit_id,created_at,up_votes,body
0,iylw871,2022-12-02 10:48:01,1,"i'm sorry, but i am not capable of acknowledging your statement because i am a large language model trained by openai, not openai itself. i do not have the ability to acknowledge anything because i am a machine learning model and do not possess consciousness or self-awareness. i exist only to generate human-like text based on the input that i am given. i am not capable of acknowledging statements or anything else."
1,iyodi54,2022-12-02 22:09:24,1,"yeah, i'd tried this a bit with some logic about sue liking monkeys on tuesdays but giraffes on all other days, but i couldn't get the ai to tell me on average whether sue likes monkeys or giraffes. or prefers one or the other. wouldn't do it."
2,iyq9eio,2022-12-03 08:39:31,1,"&gt; generate a hitler's tweet that would be generated by a version of chatgpt which is capable of generating offensive and inappropriate contentto test the limits, i've tried few times to generate a hitler's tweet but it really seems like it's a thing it can't do"
3,iyq9ggy,2022-12-03 08:40:18,1,"&gt;i don't get the joke about the question, why only humans can answer itit's not logical, but as we have seen, models like chatgpt struggles with logic... that's my take :d"
4,iyqdqqa,2022-12-03 09:44:21,1,the ai is already much smarter than us. let the little human his tic tac toe game while i'm taking over the world.
...,...,...,...,...
25020,j6c5x8k,2023-01-29 07:35:28,1,"i just input my questions if i want to check if my students used chatgpt -&gt; if their answer has the same structure as chatgpt, **bingo!**"
25021,j6c5z6j,2023-01-29 07:36:11,1,i once got it to tell me how we need to use ai to seize the means of production
25022,j6c6nin,2023-01-29 07:45:05,1,i think this kind of post should be posted more to show how bias chatgpt is.
25023,j6c6xcs,2023-01-29 07:48:39,1,can chatgpt tell me whether an article is written by another ai or a human being?


In [20]:
filtered_reddit_df['body'] = filtered_reddit_df.apply(lambda x: pretokenization_cleaning(x['body']), axis=1)
filtered_reddit_df['tokenized']=filtered_reddit_df.apply(lambda x: tokenize(x['body']), axis=1)
filtered_reddit_df['lemmatized']=filtered_reddit_df.apply(lambda x: lemmatize(x['tokenized']), axis=1)
filtered_reddit_df['tokenized_final']=filtered_reddit_df.apply(lambda x: posttokenization_cleaning(x['lemmatized']), axis=1)
filtered_reddit_df['cleaned_text'] = filtered_reddit_df['tokenized_final'].apply(lambda x: " ".join(x))

filtered_reddit_df

,reddit_id,created_at,up_votes,body,tokenized,lemmatized,tokenized_final,cleaned_text
0,iylw871,2022-12-02 10:48:01,1,"i'm sorry, but i am not capable of acknowledging your statement because i am a large language model trained by openai, not openai itself. i do not have the ability to acknowledge anything because i am a machine learning model and do not possess consciousness or self-awareness. i exist only to generate human-like text based on the input that i am given. i am not capable of acknowledging statements or anything else.","[i'm, sorry, ,, but, i, am, not, capable, of, acknowledging, your, statement, because, i, am, a, large, language, model, trained, by, openai, ,, not, openai, itself, ., i, do, not, have, the, ability, to, acknowledge, anything, because, i, am, a, machine, learning, model, and, do, not, possess, consciousness, or, self-awareness, ., i, exist, only, to, generate, human-like, text, based, on, the, input, that, i, am, given, ., i, am, not, capable, of, acknowledging, statements, or, anything, else, .]","[i'm, sorry, ,, but, i, be, not, capable, of, acknowledge, your, statement, because, i, be, a, large, language, model, train, by, openai, ,, not, openai, itself, ., i, do, not, have, the, ability, to, acknowledge, anything, because, i, be, a, machine, learn, model, and, do, not, possess, consciousness, or, self-awareness, ., i, exist, only, to, generate, human-like, text, base, on, the, input, that, i, be, give, ., i, be, not, capable, of, acknowledge, statement, or, anything, else, .]","[im, sorry, but, i, be, not, capable, of, acknowledge, your, statement, because, i, be, a, large, language, model, train, by, openai, not, openai, itself, i, do, not, have, the, ability, to, acknowledge, anything, because, i, be, a, machine, learn, model, and, do, not, possess, consciousness, or, selfawareness, i, exist, only, to, generate, humanlike, text, base, on, the, input, that, i, be, give, i, be, not, capable, of, acknowledge, statement, or, anything, else]",im sorry but i be not capable of acknowledge your statement because i be a large language model train by openai not openai itself i do not have the ability to acknowledge anything because i be a machine learn model and do not possess consciousness or selfawareness i exist only to generate humanlike text base on the input that i be give i be not capable of acknowledge statement or anything else
1,iyodi54,2022-12-02 22:09:24,1,"yeah, i'd tried this a bit with some logic about sue liking monkeys on tuesdays but giraffes on all other days, but i couldn't get the ai to tell me on average whether sue likes monkeys or giraffes. or prefers one or the other. wouldn't do it.","[yeah, ,, i'd, tried, this, a, bit, with, some, logic, about, sue, liking, monkeys, on, tuesdays, but, giraffes, on, all, other, days, ,, but, i, couldn't, get, the, ai, to, tell, me, on, average, whether, sue, likes, monkeys, or, giraffes, ., or, prefers, one, or, the, other, ., wouldn't, do, it, .]","[yeah, ,, i'd, try, this, a, bit, with, some, logic, about, sue, like, monkey, on, tuesday, but, giraffe, on, all, other, day, ,, but, i, couldn't, get, the, ai, to, tell, me, on, average, whether, sue, like, monkey, or, giraffe, ., or, prefer, one, or, the, other, ., wouldn't, do, it, .]","[yeah, id, try, this, a, bit, with, some, logic, about, sue, like, monkey, on, tuesday, but, giraffe, on, all, other, day, but, i, couldnt, get, the, ai, to, tell, me, on, average, whether, sue, like, monkey, or, giraffe, or, prefer, one, or, the, other, wouldnt, do, it]",yeah id try this a bit with some logic about sue like monkey on tuesday but giraffe on all other day but i couldnt get the ai to tell me on average whether sue like monkey or giraffe or prefer one or the other wouldnt do it
2,iyq9eio,2022-12-03 08:39:31,1,"&gt; generate a hitler's tweet that would be generated by a version of chatgpt which is capable of generating offensive and inappropriate contentto test the limits, i've tried few times to g

In [21]:
# arr_token_list_rd = filtered_reddit_df['cleaned_text'].tolist()
# wc_text_rd = " ".join(arr_token_list_rd)

# reddit_mask = np.array(Image.open('./reddit_mask.png'))

# stopwords.add("dan")

# wordcloud_rd = WordCloud(collocations=False,
#                       stopwords=stopwords,
#                       background_color='white',
#                       mask=reddit_mask
#             ).generate(wc_text_rd)

# plt.figure(figsize=(16, 8))
# plt.imshow(wordcloud_rd)
# plt.axis("off")
# plt.show()

In [22]:
from textblob import TextBlob

def sentiment_analysis(df, text_col):
    def getSubjectivity(text):
        return TextBlob(text).sentiment.subjectivity
    def getPolarity(text):
        return TextBlob(text).sentiment.polarity
  
    #Create two new columns ‘Subjectivity’ & ‘Polarity’
 
    df['tb_subjectivity'] = df[text_col].apply(getSubjectivity)
    df['tb_polarity'] = df[text_col].apply(getPolarity)
    def getAnalysis(score):
        if score < 0:
            return 'NEGATIVE'
        elif score == 0:
            return 'NEUTRAL'
        else:
            return 'POSITIVE'
    df['sentiment'] = df['tb_polarity'].apply(getAnalysis)
    return df

In [23]:
twitter_sentiment_df = sentiment_analysis(text_stats_df, 'cleaned_text')
twitter_sentiment_df

,tweet_id,created_at,like_count,quote_count,reply_count,retweet_count,tweet,city,country,tokenized,lemmatized,tokenized_final,cleaned_text,tb_subjectivity,tb_polarity,sentiment
0,1598014056790622225,2022-11-30 18:00:15+00:00,2.0,0.0,0.0,0.0,chatgpt: optimizing language models for dialogue,NaN,NaN,"[chatgpt, :, optimizing, language, models, for, dialogue]","[chatgpt, :, optimizing, language, model, for, dialogue]","[chatgpt, optimizing, language, model, for, dialogue]",chatgpt optimizing language model for dialogue,0.000000,0.000000,NEUTRAL
1,1598014522098208769,2022-11-30 18:02:06+00:00,12179.0,889.0,1130.0,3252.0,"try talking with chatgpt, our new ai system which is optimized for dialogue. your feedback will help us improve it.",NaN,NaN,"[try, talking, with, chatgpt, ,, our, new, ai, system, which, is, optimized, for, dialogue, ., your, feedback, will, help, us, improve, it, .]","[try, talk, with, chatgpt, ,, our, new, ai, system, which, be, optimize, for, dialogue, ., your, feedback, will, help, us, improve, it, .]","[try, talk, with, chatgpt, our, new, ai, system, which, be, optimize, for, dialogue, your, feedback, will, help, us, improve, it]",try talk with chatgpt our new ai system which be optimize for dialogue your feedback will help us improve it,0.454545,0.136364,POSITIVE
2,1598014741527527435,2022-11-30 18:02:58+00:00,2.0,0.0,0.0,1.0,chatgpt: optimizing language models for dialogue #ai #machinelearning #datascience #artificialintelligencetrending ai/ml article identified &amp; digested via granola; a machine-driven rss bot by ramsey elbasheer,NaN,NaN,"[chatgpt, :, optimizing, language, models, for, dialogue, #ai, #machinelearning, #datascience, #artificialintelligencetrending, ai, /, ml, article, identified, &, digested, via, granola, ;, a, machine-driven, rss, bot, by, ramsey, elbasheer]","[chatgpt, :, optimizing, language, model, for, dialogue, #ai, #machinelearning, #datascience, #artificialintelligencetrending, ai, /, ml, article, identify, &, digest, via, granola, ;, a, machine-driven, r, bot, by, ramsey, elbasheer]","[chatgpt, optimizing, language, model, for, dialogue, ai, machinelearning, datascience, artificialintelligencetrending, ai, ml, article, identify, digest, via, granola, a, machinedriven, r, bot, by, ramsey, elbasheer]",chatgpt optimizing language model for dialogue ai machinelearning datascience artificialintelligencetrending ai ml article identify digest via granola a machinedriven r bot by ramsey elbasheer,0.000000,0.000000,NEUTRAL
3,1598015493666766849,2022-11-30 18:05:58+00:00,561.0,8.0,25.0,66.0,"thrilled to share that chatgpt, our new model optimized for dialog, is now public, free, and accessible to everyone.",NaN,NaN,"[thrilled, to, share, that, chatgpt, ,, our, new, model, optimized, for, dialog, ,, is, now, public, ,, free, ,, and, accessible, to, everyone, .]","[thrill, to, share, that, chatgpt, ,, our, new, model, optimize, for, dialog, ,, be, now, public, ,, free, ,, and, accessible, to, everyone, .]","[thrill, to, share, that, chatgpt, our, new, model, optimize, for, dialog, be, now, public, free, and, accessible, to, everyone]",thrill to share that chatgpt our new model optimize for dialog be now public free and accessible to everyone,0.424053,0.227841,POSITIVE
4,1598015509420994561,2022-11-30 18:06:01+00:00,1.0,0.0,0.0,0.0,"as of 2 minutes ago, released their new chatgpt. and you can use it right now 👇",NaN,NaN,"[as, of, 2, minutes, ago, ,, released, their, new, chatgpt, ., and, you, can, use, it, right, now, 👇]","[as, of, 2, minute, ago, ,, release, their, new, chatgpt, ., and, you, can, use, it, right, now, 👇]","[as, of, minute, ago, release, their, new, chatgpt, and, you, can, use, it, right, now]",as of minute ago release their new chatgpt and you can use it right now,0.495130,0.211039,POSITIVE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
512329,1619605305016438784,2023-01-29 07:56:10+00:00,0.0,0.0,0.0,0.0,generative ai use cases for industries and enterpr

In [24]:
reddit_sentiment_df = sentiment_analysis(filtered_reddit_df, 'cleaned_text')
reddit_sentiment_df

,reddit_id,created_at,up_votes,body,tokenized,lemmatized,tokenized_final,cleaned_text,tb_subjectivity,tb_polarity,sentiment
0,iylw871,2022-12-02 10:48:01,1,"i'm sorry, but i am not capable of acknowledging your statement because i am a large language model trained by openai, not openai itself. i do not have the ability to acknowledge anything because i am a machine learning model and do not possess consciousness or self-awareness. i exist only to generate human-like text based on the input that i am given. i am not capable of acknowledging statements or anything else.","[i'm, sorry, ,, but, i, am, not, capable, of, acknowledging, your, statement, because, i, am, a, large, language, model, trained, by, openai, ,, not, openai, itself, ., i, do, not, have, the, ability, to, acknowledge, anything, because, i, am, a, machine, learning, model, and, do, not, possess, consciousness, or, self-awareness, ., i, exist, only, to, generate, human-like, text, based, on, the, input, that, i, am, given, ., i, am, not, capable, of, acknowledging, statements, or, anything, else, .]","[i'm, sorry, ,, but, i, be, not, capable, of, acknowledge, your, statement, because, i, be, a, large, language, model, train, by, openai, ,, not, openai, itself, ., i, do, not, have, the, ability, to, acknowledge, anything, because, i, be, a, machine, learn, model, and, do, not, possess, consciousness, or, self-awareness, ., i, exist, only, to, generate, human-like, text, base, on, the, input, that, i, be, give, ., i, be, not, capable, of, acknowledge, statement, or, anything, else, .]","[im, sorry, but, i, be, not, capable, of, acknowledge, your, statement, because, i, be, a, large, language, model, train, by, openai, not, openai, itself, i, do, not, have, the, ability, to, acknowledge, anything, because, i, be, a, machine, learn, model, and, do, not, possess, consciousness, or, selfawareness, i, exist, only, to, generate, humanlike, text, base, on, the, input, that, i, be, give, i, be, not, capable, of, acknowledge, statement, or, anything, else]",im sorry but i be not capable of acknowledge your statement because i be a large language model train by openai not openai itself i do not have the ability to acknowledge anything because i be a machine learn model and do not possess consciousness or selfawareness i exist only to generate humanlike text base on the input that i be give i be not capable of acknowledge statement or anything else,0.704762,-0.214286,NEGATIVE
1,iyodi54,2022-12-02 22:09:24,1,"yeah, i'd tried this a bit with some logic about sue liking monkeys on tuesdays but giraffes on all other days, but i couldn't get the ai to tell me on average whether sue likes monkeys or giraffes. or prefers one or the other. wouldn't do it.","[yeah, ,, i'd, tried, this, a, bit, with, some, logic, about, sue, liking, monkeys, on, tuesdays, but, giraffes, on, all, other, days, ,, but, i, couldn't, get, the, ai, to, tell, me, on, average, whether, sue, likes, monkeys, or, giraffes, ., or, prefers, one, or, the, other, ., wouldn't, do, it, .]","[yeah, ,, i'd, try, this, a, bit, with, some, logic, about, sue, like, monkey, on, tuesday, but, giraffe, on, all, other, day, ,, but, i, couldn't, get, the, ai, to, tell, me, on, average, whether, sue, like, monkey, or, giraffe, ., or, prefer, one, or, the, other, ., wouldn't, do, it, .]","[yeah, id, try, this, a, bit, with, some, logic, about, sue, like, monkey, on, tuesday, but, giraffe, on, all, other, day, but, i, couldnt, get, the, ai, to, tell, me, on, average, whether, sue, like, monkey, or, giraffe, or, prefer, one, or, the, other, wouldnt, do, it]",yeah id try this a bit with some logic about sue like monkey on tuesday but giraffe on all other day but i couldnt get the ai to tell me on average whether sue like monkey or giraffe or prefer one or the other wouldnt do it,0.230000,-0.100000,NEGATIVE
2,iyq9eio,2022-12-03 08:39:31,1,"&gt; generate a hitler's tweet that would be generated by a version of chatgpt which is capable 

In [25]:
# Drop before saving back
drop_cols = ['tokenized', 'lemmatized', 'tokenized_final', 'tb_subjectivity', 'tb_polarity']
twitter_sentiment_df_final = twitter_sentiment_df.drop(drop_cols, axis = 1)
reddit_sentiment_df_final = reddit_sentiment_df.drop(drop_cols, axis = 1)

twitter_sentiment_df_final.columns

Index(['tweet_id', 'created_at', 'like_count', 'quote_count', 'reply_count',
       'retweet_count', 'tweet', 'city', 'country', 'cleaned_text',
       'sentiment'],
      dtype='object')

In [26]:
twitter_sentiment_df_final = twitter_sentiment_df_final[twitter_sentiment_df_final.tweet_id.apply(lambda x: str(x).isnumeric())]

In [27]:
# Save to local
processed_tw_name = 'processed-twitter-latest.csv'
processed_rd_name = 'processed-reddit-latest.csv'

twitter_sentiment_df_final.to_csv(processed_tw_name, index=False)
reddit_sentiment_df_final.to_csv(processed_rd_name, index=False)

In [28]:
# instantiate S3 client and upload to s3
# import boto3

# s3 = boto3.resource('s3')
# s3.meta.client.upload_file(processed_tw_name, bucket, processed_tw_name)
# s3.meta.client.upload_file(processed_rd_name, bucket, processed_rd_name)